In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pylab as py

# Mostly from the very begining of the semester where I generally investigate the data set given to me

# not particularly useful to the current analysis

In [ ]:
#list of event names to extract data for each event from the h5 file
events = []
for i in range(1,75):
    events.append('get/'+str(i))

# Event 1:
# Plot time vs charge for each pad to look for peaks with chage > 500
x                  : 1d array to hold time values from 0 through 511

y                  : 1d array to hold charge values for each pad

charge_foreach_pad : (10240,512) array to hold arrays of charge values for each pad

In [ ]:
charge_foreach_pad = np.ndarray(10240, dtype=object)    #list to hold arrays of charge for each pad
pads_witha_hit = np.zeros(10240)                        #array to have a value of 1 for pad(index) with a "hit" (charge>500)
with h5py.File('corrected_run_0210.h5', 'r') as f:
    #===================== for event 1 ================================
    dset = f['get/1']                                   #dset.shape (10240, 517) corresping to (pads, time)
    #dset = dset[:,6:517]                               #removes first 5 columns
    dset = dset[:,10:510]                           
    for i in range(len(dset)):                          #i from 0 to 10240 for each pad
        pad = dset[i]                                   #select pad from dset 
        y = np.zeros(len(dset[0]))#512)                 #[512] array to hold charge values
        for j in range(len(dset[0])):                   #j from 0 to 512 for time values
            if pad[j] >= 500:                           #if charge at time value is above 500
                pads_witha_hit[i] = 1
            y[j] = pad[j]
        charge_foreach_pad[i] = y                       #add pads charge values to charge_perpad list
    
x = np.arange(len(dset[0]))                                      #array to hold time values

## For each pad and if it has a "hit" check if the charge is > 800, if so make value 2

In [ ]:
for i in range(10240):
    if pads_witha_hit[i] == 1:                   #if pads_witha_hit has a value of 1 then it is a pad with charge > 500
        pad = charge_foreach_pad[i]              
        for j in range(len(dset[0])):#512):                  
            if pad[j] > 800:                     #if the pad has a charge over 800 
               pads_witha_hit[i] = 2             #make the value 2 so that the plot will have a different title

## For each pad if there is a hit, plot the pad

In [ ]:
num_of_plots = 0
for i in range(10240):
    if pads_witha_hit[i] != 0:                            #if the value of pads_witha_hit for each pad is non zero (1 or 2)
        num_of_plots += 1
        if pads_witha_hit[i] == 1:                        #if the value == 1 it does not have charge > 800
            py.plot(x,charge_foreach_pad[i])
            py.title("Pad "+str(i))                       #regular title
        else:                                             #if the value is 2 then it has a charge > 800
            py.plot(x,charge_foreach_pad[i], 'r')
            py.title("pad "+str(i)+" and charge value exceeds 800") #add this to the title
        py.xlabel("Time values")
        py.ylabel("charge values")
        py.ylim(200,800)
        py.show()
print(num_of_plots)

## Plot only the pads with well defined peaks

In [ ]:
peaks=pd.read_csv('padswithahitevent1.csv')             #manually picked out well defined peaks
peak_padnum = peaks['Pad Numbers']                      #array of pad nums showing defined peaks

In [ ]:
x = range(1, 501)
for i in range(len(peak_padnum)):
    if peak_padnum[i] == 5771:
        py.plot(x, charge_foreach_pad[peak_padnum[i]])   
        py.title("Charge Trace for Pad "+str(peak_padnum[i]))       
        py.xlabel("Time Buckets")
        py.ylabel("Charge Values")
        py.ylim(200,800)
        py.show()

In [ ]:
for i in range(len(peak_padnum)):
    if pads_witha_hit[peak_padnum[i]] != 0:             #if the value of pads_witha_hit for each pad is non zero (1 or 2)
        if pads_witha_hit[peak_padnum[i]] == 1:         #if the value == 1 it does not have charge > 800
            py.plot(x,charge_foreach_pad[peak_padnum[i]])   #time vs charge for the number of pads given
            py.title("Pad "+str(peak_padnum[i]))        #regular title
        else:                                               #if the value is 2 then it has a charge > 800
            py.plot(x,charge_foreach_pad[peak_padnum[i]], 'r')   #time vs charge for the number of pads given
            py.title("pad "+str(peak_padnum[i])+" | charge value exceeds 800") #add this to the title
        py.xlabel("Time values")
        py.ylabel("charge values")
        py.ylim(200,800)
        py.show()

# Plot x vs y coordinates of pads with a "hit" (>500) using the look up table

-col E has value 0 on row 2676 of look up table

-the columns of the look up table are CoBo, AsAd, AGET, channel and pad number

# histograms of charge values
## not informative

# ==============================================================

# For Event 1 when charge value > 500

## To gather what we consider to be "hits"

In [ ]:
hit_dsetholder_500 = []                          #holds np arrays of copies of dset only containing zeros and "hits" 
counterSet_500 = []                              #tells us how many "hits" for the assumed threshold
with h5py.File('run_0210.h5', 'r') as f:
    #===================== for event 1 ================================
    dset = f['get/1']                             #dset.shape (10240, 517) corresping to (pads, time)
    dset = dset[:,6:517]                          #removes first 5 columns
    hit_dset_500 = np.zeros(shape=dset.shape)     #same shape as dset
    for i in range(len(dset)):                    #i from 0 to 10240 for each pad
        pad = dset[i]                             #select pad from dset 
        counter = 0                               #initiate counter for each pad
        for j in range(len(dset[0])):             #j from 0 to 512 for time values
            if pad[j] >= 500:                     #if charge at time value is above 500
                counter += 1                      #increase counter by one for the pad
                hit_dset_500[i,j] = pad[j]        #add value to hit_dset
                
        counterSet_500.append(counter)
        hit_dsetholder_500.append(hit_dset_500)

### To plot a histagram of charge value > 500

In [ ]:
hits_500 = []
for i in range(len(hit_dset_500)):
    for j in range(len(hit_dset_500[0])):
        if hit_dset_500[i,j] > 1:
            hits_500.append(hit_dset_500[i,j])
print(len(hits_500))

In [ ]:
def hist_event1_over500():
    plt.hist(hits_500, bins=100, range=[400,800])
    plt.xlabel("Charge Value > 500")
    plt.ylabel("Frequency")
    plt.title('Distribution of charge > 500 for event 1')
    plt.rcParams['figure.figsize'] = [25, 15] #changes size of plot
    plt.show()

In [ ]:
hist_event1_over500()

In [ ]:
print(len(hit_dset_500[0]))
print(len(hit_dset_500))

### Need the look up table to be able to plot x vs y to look for spirals

# Note: Sadly NOT True

In [ ]:
x = np.zeros(shape=511)
y = np.zeros(shape=10240)
#for i in range(len(hit_dset_500[0])):
    #for j in range(len(hit_dset_500)):
        #if hit_dset_500[j,i] >= 500:
            #x[i],y[j] = 1,1
            
#py.plot(x,y,"k.")

# For Event 1 when charge value > 475

In [ ]:
#hit_dsetholder = np.zeros(shape=[1,74])
#counterSet = np.zeros(shape=[1,10240])
hit_dsetholder_475 = []
counterSet_475 = []
with h5py.File('run_0210.h5', 'r') as f:
    #===================== for event 1 ================================
    dset = f['get/1'] #dset.shape (10240, 517) corresping to (pads, time)
    dset = dset[:,6:517] #removes first 5 columns
    hit_dset_475 = np.zeros(shape=dset.shape) #same shape as dset
    for i in range(len(dset)): #i from 0 to 10240 for each pad
        pad = dset[i] #select pad from dset 
        counter = 0 #initiate counter for each pad
        for j in range(len(dset[0])): #j from 0 to 512 for time values
            if pad[j] >= 475: #if charge at time value is above 475
                counter += 1  #increase counter by one for the pad
                hit_dset_475[i,j] = pad[j] #add value to hit_dset
                
        #counterSet[i] = counter #add count value of hits on pad to counterSet 
        counterSet_475.append(counter)
        #hit_dsetholder[0] = hit_dset #add hit_dset to the hit_dsetholder list to store the values
        hit_dsetholder_475.append(hit_dset_475)   

In [ ]:
#print(len(maxValuesForAllEvents)) #min value: 204      max value: 4095
#print(hit_dsetholder[0])
#print(hit_dsetholder[0]) #data rate exceeded
#for i in range(len(counterSet)):
    #print('Number of hits on pad '+str(i)+': '+str(counterSet[i]))
#print(hit_dset[69])
hits_475 = []
for i in range(len(hit_dset_475)):
    for j in range(len(hit_dset_475[0])):
        if hit_dset_475[i,j] > 1:
            hits_475.append(hit_dset_475[i,j])
print(len(hits_475))

In [ ]:
def hist_event1_over475():
    plt.hist(hits_475, bins=100, range=[400,800])
    plt.xlabel("Charge Value > 475")
    plt.ylabel("Frequency")
    plt.title('Distribution of charge > 475 for event 1')
    plt.rcParams['figure.figsize'] = [25, 15] #changes size of plot
    plt.show()

In [ ]:
hist_event1_over475()

# For Event 1 when charge value > 450

In [ ]:
hit_dsetholder_450 = []
counterSet_450 = []
with h5py.File('run_0210.h5', 'r') as f:
    #===================== for event 1 ================================
    dset = f['get/1'] #dset.shape (10240, 517) corresping to (pads, time)
    dset = dset[:,6:517] #removes first 5 columns
    hit_dset_450 = np.zeros(shape=dset.shape) #same shape as dset
    for i in range(len(dset)): #i from 0 to 10240 for each pad
        pad = dset[i] #select pad from dset 
        counter = 0 #initiate counter for each pad
        for j in range(len(dset[0])): #j from 0 to 512 for time values
            if pad[j] >= 450: #if charge at time value is above 475
                counter += 1  #increase counter by one for the pad
                hit_dset_450[i,j] = pad[j] #add value to hit_dset
                
        counterSet_450.append(counter)
        hit_dsetholder_450.append(hit_dset_450) 

In [ ]:
hits_450 = []
for i in range(len(hit_dset_450)):
    for j in range(len(hit_dset_450[0])):
        if hit_dset_450[i,j] > 1:
            hits_450.append(hit_dset_450[i,j])
print(len(hits_450))

In [ ]:
def hist_event1_over450():
    plt.hist(hits_450, bins=100, range=[400,700])
    plt.xlabel("Charge Value > 450")
    plt.ylabel("Frequency")
    plt.title('Distribution of charge > 450 for event 1')
    plt.rcParams['figure.figsize'] = [25, 15] #changes size of plot
    plt.show()

In [ ]:
hist_event1_over450()

# For Event 1 when charge value > 425

In [ ]:
hit_dsetholder_425 = []
counterSet_425 = []
with h5py.File('run_0210.h5', 'r') as f:
    #===================== for event 1 ================================
    dset = f['get/1'] #dset.shape (10240, 517) corresping to (pads, time)
    dset = dset[:,6:517] #removes first 5 columns
    hit_dset_425 = np.zeros(shape=dset.shape) #same shape as dset
    for i in range(len(dset)): #i from 0 to 10240 for each pad
        pad = dset[i] #select pad from dset 
        counter = 0 #initiate counter for each pad
        for j in range(len(dset[0])): #j from 0 to 512 for time values
            if pad[j] >= 425: #if charge at time value is above 475
                counter += 1  #increase counter by one for the pad
                hit_dset_425[i,j] = pad[j] #add value to hit_dset
                
        counterSet_425.append(counter)
        hit_dsetholder_425.append(hit_dset_425)

In [ ]:
hits_425 = []
for i in range(len(hit_dset_425)):
    for j in range(len(hit_dset_425[0])):
        if hit_dset_425[i,j] > 1:
            hits_425.append(hit_dset_425[i,j])
print(len(hits_425))

In [ ]:
def hist_event1_over425():
    plt.hist(hits_425, bins=100, range=[400,700])
    plt.xlabel("Charge Value > 425")
    plt.ylabel("Frequency")
    plt.title('Distribution of charge > 425 for event 1')
    plt.rcParams['figure.figsize'] = [25, 15] #changes size of plot
    plt.show()

In [ ]:
hist_event1_over425()

# For all events when charge value > 500

In [ ]:
hit_dsetholder_all = []
counterSet_all = []

with h5py.File('run_0210.h5', 'r') as f:
    #===================== for all events ================================
    for event in events:                                   #event out of the 0 to 74 events (strings stored in the list events)
        dset = f[event]                                    #dset for selected event
        dset = dset[:,6:517]                               #removes first 5 columns
        hit_dset_all = np.zeros(shape=dset.shape)          #same shape as dset
        for i in range(len(dset)):                         #i from 0 to 10240 for each pad
            pad = dset[i]                                  #select pad from dset 
            counter = 0                                    #initiate counter for each pad
            for j in range(len(dset[0])):                  #j from 0 to 512 for time values
                if pad[j] >= 500:                          #if charge at time value is above 500
                    counter += 1                           #increase counter by one for the pad
                    hit_dset_all[i,j] = pad[j]             #add value to hit_dset
            counterSet_all.append(counter)                 #add count value of hits on pad to counterSet 
            hit_dsetholder_all.append(hit_dset_all)        #add hit_dset to the hit_dsetholder list to store the values

In [ ]:
hits_all = []
for i in range(len(hit_dset_all)):
    for j in range(len(hit_dset_all[0])):
        if hit_dset_all[i,j] > 1:
            hits_all.append(hit_dset_all[i,j])
print(len(hits_all))

In [ ]:
def hist_all():
    plt.hist(hits_all, bins=100, range=[400,700])
    plt.xlabel("Charge Value > 500")
    plt.ylabel("Frequency")
    plt.title('Distribution of charge > 500 for all events')
    plt.rcParams['figure.figsize'] = [25, 15] #changes size of plot
    plt.show()

In [ ]:
hist_all()

In [ ]:
def histogram(eventNumber):
    plt.hist(maxValuesPerEvent[eventNumber-1], bins=100, range=[200,700])
    plt.xlabel("Max Charge Value")
    plt.ylabel("Frequency")
    plt.title('Distribution of max values of charge on each pad for event '+str(eventNumber))
    plt.rcParams['figure.figsize'] = [25, 15] #changes size of plot
    plt.show()

In [ ]:
histogram(1)

In [ ]:
histogram(2)